# LLaMA 사용하기

In [ ]:
!pip install transformers accelerate

- pytorch의 설치 방법은 pytorch 공식 홈페이지를 참고
  - https://pytorch.org/

In [ ]:
!conda install -y pytorch::pytorch -c pytorch

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
import transformers
import torch

# https://huggingface.co/meta-llama/Meta-Llama-3-8B
# 16GB이상의 VRAM 필요
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_id= "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

In [3]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    # device_map="auto", # NVIDIA GPU가 있을 경우 CUDA를 사용하도록 세팅됨
    device_map="mps", # M시리즈(M1, M1 pro, M2, M3 등) mac인 경우 manual하게 mps로 세팅
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
pipeline.torch_dtype

Llama3의 prompt format
- https://llama.meta.com/docs/model-cards-and-prompt-formats/meta-llama-3/

In [4]:
messages = [
    {"role": "system", "content": "너는 도움이 되는 어시스턴트이다."},
    {"role": "user", "content": "안녕이 영어로 뭐야?"},
]

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [5]:
%%time
outputs = pipeline(
    messages,
    max_new_tokens=24,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


CPU times: user 2.58 s, sys: 832 ms, total: 3.41 s
Wall time: 6.1 s


In [6]:
print(outputs[0]["generated_text"][-1])

{'role': 'assistant', 'content': '😊\n\n"안녕" is translated to "Hello" in English. However, it\'s worth noting that "'}


In [7]:
outputs

[{'generated_text': [{'role': 'system', 'content': '너는 도움이 되는 어시스턴트이다.'},
   {'role': 'user', 'content': '안녕이 영어로 뭐야?'},
   {'role': 'assistant',
    'content': '😊\n\n"안녕" is translated to "Hello" in English. However, it\'s worth noting that "'}]}]

## 참고

GPU를 장착한 Linux 머신이 있다면 더 빠른 추론을 위해 `vllm`이라고 하는 LLM 서빙 엔진을 사용 할 수 있다.
- vllm
  - https://github.com/vllm-project/vllm